In [1]:
from pathlib import Path
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import lit,to_timestamp, coalesce, col, date_format, when
from functools import reduce
import csv
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import dtale
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


spark = SparkSession.builder.getOrCreate()

2025-04-04 13:06:43.720592: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-04 13:06:43.721785: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-04 13:06:43.724431: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-04 13:06:43.732635: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743790003.747539  188699 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743790003.75

In [3]:
DFP_AQ2_CPM10 = pd.read_csv("DFP_AQ2_CPM10_RR_IN.csv", index_col=0, parse_dates=True)
DFP_AQ2_CPM2_5 = pd.read_csv("DFP_AQ2_CPM2_5_RR_IN.csv", index_col=0, parse_dates=True)
DFP_AQ1_CPM10 = pd.read_csv("DFP_AQ1_CPM10_RR_IN.csv", index_col=0, parse_dates=True)
DFP_AQ1_CPM2_5 = pd.read_csv("DFP_AQ1_CPM2_5_RR_IN.csv", index_col=0, parse_dates=True)


In [6]:
DFP_AQ2_CPM10

,pm2_5,pm10,co,o3,humidity,temperature
time_format,,,,,,
2023-08-01 00:00:00,9.0,0.156250,15.5,1.0,66.25,13.30
2023-08-01 01:00:00,6.0,0.109375,16.0,1.0,68.30,12.90
2023-08-01 02:00:00,5.0,0.093750,14.5,1.0,69.70,12.90
2023-08-01 03:00:00,7.0,0.125000,14.0,1.0,70.50,12.65
2023-08-01 04:00:00,9.5,0.164062,14.5,1.0,72.40,12.40
...,...,...,...,...,...,...
2024-03-14 20:00:00,NaN,0.698638,NaN,NaN,NaN,NaN
2024-03-14 21:00:00,NaN,0.682091,NaN,NaN,NaN,NaN
2024-03-14 22:00:00,NaN,0.690630,NaN,NaN,NaN,NaN
